In [18]:
import gensim
from gensim import corpora
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd


In [25]:
df = pd.read_csv('E:\\studies\\USJ FOT\\lecture\\Research\\CodeBase\\CreatingDataset\\cleaned\\acroeconomic_classifier_dataset_4_hypercleaned.csv'
                 )
# Convert all sentences to strings and handle NaN values
df['sentence'] = df['sentence'].astype(str).fillna('')

In [23]:
# Download stopwords from NLTK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
# Preprocess sentences
stop_words = set(stopwords.words('english'))
texts = [[word for word in word_tokenize(sentence.lower()) if word.isalnum() and word not in stop_words] for sentence in df['sentence']]

In [29]:
# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train LDA model
lda_model = gensim.models.LdaModel(corpus, num_topics=7, id2word=dictionary, passes=15)

# Print topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")


Topic: 0 
Words: 0.016*"policy" + 0.015*"price" + 0.011*"government" + 0.010*"term" + 0.010*"expect" + 0.010*"measure" + 0.010*"export" + 0.009*"economy" + 0.009*"level" + 0.009*"year"

Topic: 1 
Words: 0.019*"policy" + 0.017*"inflation" + 0.016*"rate" + 0.016*"bank" + 0.015*"price" + 0.014*"central" + 0.012*"monetary" + 0.011*"market" + 0.010*"term" + 0.010*"government"

Topic: 2 
Words: 0.015*"sector" + 0.015*"foreign" + 0.015*"bank" + 0.013*"market" + 0.011*"credit" + 0.010*"central" + 0.010*"month" + 0.010*"government" + 0.009*"financial" + 0.009*"expect"

Topic: 3 
Words: 0.047*"debt" + 0.017*"imf" + 0.016*"restructuring" + 0.015*"economic" + 0.012*"eff" + 0.012*"sri" + 0.012*"programme" + 0.012*"government" + 0.011*"lanka" + 0.011*"support"

Topic: 4 
Words: 0.018*"sector" + 0.013*"inflation" + 0.010*"level" + 0.010*"government" + 0.009*"public" + 0.009*"reform" + 0.008*"policy" + 0.008*"sri" + 0.007*"lanka" + 0.007*"benefit"

Topic: 5 
Words: 0.035*"cent" + 0.026*"year" + 0.018*

In [30]:
# Assign the most likely topic to each sentence
def get_most_likely_topic(bow):
    topic_probabilities = lda_model.get_document_topics(bow)
    most_likely_topic = max(topic_probabilities, key=lambda x: x[1])[0]
    return most_likely_topic

df['lda_label'] = [get_most_likely_topic(dictionary.doc2bow(text)) for text in texts]

# Save the updated dataframe to a new CSV file
df.to_csv('E:\\studies\\USJ FOT\\lecture\\Research\\CodeBase\\CreatingDataset\\cleaned\\acroeconomic_classifier_dataset_4_hypercleaned_with_lda_labels.csv', index=False)